In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import reprlib
import math
import numbers
import itertools
import fractions
from array import array
from operator import xor
from functools import reduce


class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'

        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    def __bytes__(self):
        return bytes(bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other):
        if isinstance(other, type(self)):
            return len(self) == len(other) \
                and all(a == b for a, b in zip(self, other))

        return NotImplemented

    def __hash__(self):
        return reduce(xor, (hash(x) for x in self._components), 0)

    def __abs__(self):
        # return math.sqrt(sum(x * x for x in self))
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            raise TypeError(f'{cls.__name__} indices must be integers')

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = self.shortcut_names.find(name)
            if pos != -1:
                return self._components[pos]

        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def __setattr__(self, name, value):
        cls = type(self)
        err = ''
        if len(name) == 1:
            if name in self.shortcut_names:
                err = f'readonly attribute {name!r}'
            elif name.islower():
                err = f"can't set attributes 'a' to 'z' in {cls.__name__!r}"

        if err:
            raise AttributeError(err)

        return super().__setattr__(name, value)

    def __neg__(self):
        cls = type(self)
        return cls(-x for x in self)

    def __pos__(self):
        cls = type(self)
        return cls(self)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(x * scalar for x in self)

        return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar

    def __matmul__(self, other):
        try:
            assert len(self) == len(other)
            return sum(a * b for a, b in zip(self, other))
        except AssertionError:
            raise ValueError('length of two operands should be equal')
        except TypeError:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

    @classmethod
    def from_bytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))


"""

>>> v1 = Vector((3, 4, 5))
>>> v2 = Vector((6, 7, 8))
>>> v1 + v2
Vector([9.0, 11.0, 13.0])
>>> v1 + v2 == Vector([3 + 6, 4 + 7, 5 + 8])
True
>>> v1 @ v2
86.0
>>> v1 = Vector((3, 4, 5, 6))
>>> v2 = Vector((1, 2))
>>> v1 + v2
Vector([4.0, 6.0, 5.0, 6.0])
>>> [1.0, 2.0] + v1
Vector([4.0, 6.0, 5.0, 6.0])
>>> v1 * 10
Vector([30.0, 40.0, 50.0, 60.0])
>>> fractions.Fraction(1, 3) * v1
Vector([1.0, 1.3333333333333333, 1.6666666666666665, 2.0])
>>> v1 @ v2
Traceback (most recent call last):
    ...
ValueError: length of two operands should be equal
>>> va = Vector((3, 4, 5))
>>> vb = Vector((3, 4, 5))
>>> va == vb
True
>>> va == (3, 4, 5)
False
"""

doctest.testmod()

TestResults(failed=0, attempted=16)